In [31]:
from summary_eval.data import summary_df, prompts_df
from typing import List
import re
from autocorrect import Speller
import nltk

In [6]:
def pre_process_whitespace(text: str) -> str:
    # Replace all whitespace with a single space
    text = ' '.join(text.split())
    
    # Remove leading and trailing whitespace
    text = text.strip()
    return text

In [7]:
pre_process_whitespace(summary_df["text"].iloc[0])

'The third wave was an experimentto see how people reacted to a new one leader government. It gained popularity as people wanted to try new things. The students follow anything that is said and start turning on eachother to gain higher power. They had to stop the experement as too many people got to radical with it blindly following there leader'

In [14]:
def split_quotations(text: str) -> (str, List[str]):
    quotations = re.findall('"([^"]*)"', text)
    no_quote_text = text
    for quotation in quotations:
        no_quote_text = no_quote_text.replace(f'"{quotation}"', "")
    return no_quote_text, quotations

In [20]:
split_quotations(summary_df["text"].iloc[0])

('The third wave was an experimentto see how people reacted to a new one leader government. It gained popularity as people wanted to try new things. The students follow anything that is said and start turning on eachother to gain higher power. They had to stop the experement as too many people got to radical with it blindly following there leader',
 [])

In [18]:
split_quotations(summary_df["text"].iloc[2])

("In Egypt, there were many occupations and social classes involved in day-to-day living. In many instances if you were at the bottom of the social ladder you could climb up, you didn't have to stay a peasant you could work to bring your status up. Everyone worshipped the gods Ra, Osiris, and Isis, but also they would worship their pharaohs like gods as well. Under the pharaohs were the priests, they had the responsibility to entertain or please the said god. The Chain of Command was placed to keep everyone in check, not one person could handle all the civilians and treasures without any aid. Like the tax collector, called a vizier like stated they were in charge of collecting the peoples' tax. They were also one of the rare instances who were able to read and write, that's how they were granted  Also the soldiers did many things as they would fight in wars or . They were in charge of getting the slaves, farmers, and peasants to build palaces or the famous ancient pyramids. More skille

In [25]:
split_quotations(summary_df["text"].iloc[4])

("The Third Wave developed  rapidly because the students genuinly believed that it was the best course of action. Their grades, acomplishments, and classparticipation/ behavior had improved dramatically since the experiment began. There did not seem to be any consiquenses in the students eyes. They became extremely engaged in all the Third Wave activites both inside and outside tha classroom. The experiment ended because the students were so patriotic about the . The history class of thirty rapidly grew to 200 in three days.  That means 170 students joined a school  in two days. Thats 85 people per day! On the fifth and final day all the students had completley believed that the  was a movement that would expell democracy. They believed a candidate from the  would anounce its existance on television after five days of its success. The creater, Ron Jones, believed it had gone too far and for everyone's safety he shut it down.  If he hadn't the fake organization would have grown into som

In [ ]:
def correct_spelling(text: str) -> (str, int):
    spell = Speller(lang='en')
    count = 0
    text = []
    for word in nltk.word_tokenize(text):
        if word not in spell.known(word):
            count += 1
            text.append(spell(word))

    return spell(text)

In [34]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

2024-03-07 20:01:37,124 - INFO - Unzipping C:\Users\theaw\AppData\Local\Temp\tmp6he6izxs.zip to C:\Users\theaw\.cache\language_tool_python.
2024-03-07 20:01:41,593 - INFO - Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to C:\Users\theaw\.cache\language_tool_python.


15

In [37]:
text = pre_process_whitespace(summary_df["text"].iloc[4])
matches = tool.check(text)
len(matches)

14

In [52]:
def count_dict_field(matches: dict, field: str) -> dict:
    issue_counts = {}
    for match in matches:
        if getattr(match, field) in issue_counts:
            issue_counts[getattr(match, field)] += 1
        else:
            issue_counts[getattr(match, field)] = 1
    return issue_counts

def merge_dicts_additive(dict1: dict, dict2: dict) -> dict:
    merged = dict1
    for key in dict2:
        if key in merged:
            merged[key] += dict2[key]
        else:
            merged[key] = dict2[key]
    return merged

In [54]:
from tqdm import tqdm

categories = {}
issues = {}

for row in tqdm(summary_df.iterrows()):
    text = pre_process_whitespace(row[1]["text"])
    matches = tool.check(text)
    categories = merge_dicts_additive(categories, count_dict_field(matches, "category"))
    issues = merge_dicts_additive(issues, count_dict_field(matches, "ruleIssueType"))

categories, issues

7165it [08:32, 13.99it/s]


({'TYPOS': 13552,
  'REDUNDANCY': 436,
  'PUNCTUATION': 2015,
  'TYPOGRAPHY': 2129,
  'STYLE': 283,
  'GRAMMAR': 1057,
  'CASING': 2063,
  'MISC': 377,
  'CONFUSED_WORDS': 247,
  'NONSTANDARD_PHRASES': 8,
  'COMPOUNDING': 25,
  'COLLOCATIONS': 27,
  'AMERICAN_ENGLISH_STYLE': 11,
  'SEMANTICS': 1},
 {'misspelling': 14173,
  'style': 730,
  'typographical': 3888,
  'whitespace': 2091,
  'grammar': 1084,
  'duplication': 143,
  'uncategorized': 121,
  'inconsistency': 1})

In [51]:
display(count_dict_field(matches, "ruleIssueType"))
display(count_dict_field(matches, "category"))

{'misspelling': 14}

{'TYPOS': 14}

In [39]:
matches[0]

Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'message': 'Possible spelling mistake found.', 'replacements': ['genuinely'], 'offsetInContext': 43, 'context': '... developed rapidly because the students genuinly believed that it was the best course of...', 'offset': 54, 'errorLength': 8, 'category': 'TYPOS', 'ruleIssueType': 'misspelling', 'sentence': 'The Third Wave developed rapidly because the students genuinly believed that it was the best course of action.'})